In [1]:
import torch.nn as nn
import torch
from Seq2SeqModel.Seq2SeqModel import Seq2SeqModel
import pandas as pd
from transformers import AutoTokenizer
import torch.nn.init as init

# CSV 파일에서 데이터 불러오기
df_train = pd.read_csv("Datatset2/train.csv")
df_val = pd.read_csv("Datatset2/val.csv")
tokenizer= AutoTokenizer.from_pretrained("Datatset/tokenizer")

def str_to_list(input_string):
    cleaned_string = input_string.strip("[]").replace(" ", "")

    number_strings = cleaned_string.split(",")
    number_list = [int(num) for num in number_strings]
    return number_list

# 데이터 추출
X_train = df_train["X_train"].values.tolist()
X_train=[str_to_list(i) for i in X_train]
y_train = df_train["y_train"].values.tolist()
y_train=[str_to_list(i) for i in y_train]


X_val = df_val["X_val"].values.tolist()
X_val=[str_to_list(i) for i in X_val]
y_val = df_val["y_val"].values.tolist()
y_val=[str_to_list(i) for i in y_val]




In [2]:
device = torch.device('cpu')

tokenizer= AutoTokenizer.from_pretrained("Datatset/tokenizer")
model=None
class_weights = torch.ones(tokenizer.vocab_size+1).to(device=device)
class_weights[tokenizer.pad_token_id]=0.1
class_weights[tokenizer.pad_token_id-1]=0.1
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.01,weight=class_weights)
optimizer=None
batchsize=64
lr=0.005
print("Set")
module=Seq2SeqModel( tokenizer,loss_fn,lr,
                    X_train,y_train,X_val,y_val,batch_size=batchsize,device=torch.device("cpu"))
print(module.config.n_dec_seq)


Set
Using cpu
128


In [3]:

print("train")
module.load_model("saved56.pth")

train


In [4]:
print(module.config)

{'n_enc_vocab': 59515, 'n_dec_vocab': 59515, 'n_enc_seq': 128, 'n_dec_seq': 128, 'n_layer': 4, 'd_hidn': 128, 'i_pad': 0, 'd_ff': 256, 'n_head': 3, 'd_head': 128, 'dropout': 0.2, 'layer_norm_epsilon': 1e-12}


In [5]:
(a,b)=(torch.tensor(X_train[4]),torch.tensor(y_train[4]))

In [6]:
print(a,b,tokenizer.pad_token_id)

tensor([59514,   135,   607,  2054,     2,  3482,    10,  2843, 21048,    26,
           67,   478,     0]) tensor([59514,  4322,     5, 30508,     2,     5,  4403,    11,     5,  6676,
           27,    66,   478,     0]) 59513


In [7]:
out,*inputs=module.seq_to_seq_process(a,b)

seq_padd:  128
seq_padd:  128
seq_padd:  127


In [8]:
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens([i for i in inputs[0] if i!=tokenizer.pad_token_id])))
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens([i for i in inputs[1] if i!=tokenizer.pad_token_id])))

<bos> Sentence,▁word and▁letter▁counts for this document</s>
<bos> Nombre de phrases, de mots et de lettres pour ce document


In [9]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens([i for i in out if i!=tokenizer.pad_token_id]))

'& &'